In the pervious file, I have explored two different machine learning methods with Sklearn and compared the results. 
In the following sections, I aim to use deep learning to predict the car price and, in the end, compare the results with the machine learning methods.

In [1]:
#upload libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# let's upload the preprocessed data from the last project

In [2]:
data = pd.read_csv('data_preprocessed.csv')
data.head()

,Unnamed: 0,Mileage,EngineV,Price_log,Brand_Audi,Brand_BMW,Brand_Mercedes-Benz,Brand_Mitsubishi,Brand_Renault,Brand_Toyota,...,Body_other,Body_sedan,Body_vagon,Body_van,Engine Type_Diesel,Engine Type_Gas,Engine Type_Other,Engine Type_Petrol,Registration_no,Registration_yes
0,0,277,2.0,8.342840,0,1,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
1,1,427,2.9,8.974618,0,0,1,0,0,0,...,0,0,0,1,1,0,0,0,0,1
2,2,358,5.0,9.495519,0,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,1
3,3,240,4.2,10.043249,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4,4,120,2.0,9.814656,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1


In [3]:
#let's drop unnamed column.
data = data.drop(['Unnamed: 0'], axis = 1)
data.head()

,Mileage,EngineV,Price_log,Brand_Audi,Brand_BMW,Brand_Mercedes-Benz,Brand_Mitsubishi,Brand_Renault,Brand_Toyota,Brand_Volkswagen,...,Body_other,Body_sedan,Body_vagon,Body_van,Engine Type_Diesel,Engine Type_Gas,Engine Type_Other,Engine Type_Petrol,Registration_no,Registration_yes
0,277,2.0,8.342840,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
1,427,2.9,8.974618,0,0,1,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
2,358,5.0,9.495519,0,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
3,240,4.2,10.043249,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4,120,2.0,9.814656,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1


In [4]:
x = data.drop(['Price_log'], axis = 1)
y = data['Price_log']
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x) 
# scaling the data

StandardScaler()

In [5]:
# now , wee need to store them in a new variable
x_scaled = scaler.transform(x)

In [6]:
from sklearn.model_selection import train_test_split

# Split the variables with an 80% for training set and 20% for test dataset.
# To have the same split as mine, use random_state = 42
# shufelling is on by default.
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)

### Deep Learning with TensorFlow

In [7]:
import tensorflow as tf

2022-10-24 15:52:55.761278: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-24 15:52:55.761300: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [10]:
# creating a simple neural network.

# Set random seed
tf.random.set_seed(42)

# Create a  model_1
model_1 = tf.keras.Sequential([
  tf.keras.layers.Dense(128)
])

# Compile the model
model_1.compile(loss=tf.keras.losses.mae,
                        optimizer=tf.keras.optimizers.SGD(),
                        metrics=['mae'])

# Fit the model
model_1.fit(x_train, y_train, epochs=100, verbose=0)

In [12]:
model_1.evaluate(x_train,y_train)

99/99 [==============================] - 0s 1ms/step - loss: 8.6098 - mae: 8.6098


[8.609797477722168, 8.609797477722168]

In [11]:
model_1.evaluate(x_test,y_test)

25/25 [==============================] - 0s 1ms/step - loss: 8.6362 - mae: 8.6362


[8.636229515075684, 8.636229515075684]

## let's improving the model to reduce the loss function.

### step number one increasing the number of epochs from 100 to 500

In [32]:
# creating a simple neural network.

# Set random seed
tf.random.set_seed(42)

# Create a  model_2
model_2 = tf.keras.Sequential([
  tf.keras.layers.Dense(128)
])

# Compile the model
model_2.compile(loss=tf.keras.losses.mae,
                        optimizer=tf.keras.optimizers.SGD(),
                        metrics=['mae'])

# Fit the model
model_2.fit(x_train, y_train, epochs=500, verbose=0)

In [33]:
#let's evaluate the model
model_2.evaluate(x_train,y_train)

99/99 [==============================] - 0s 978us/step - loss: 5.5114 - mae: 5.5114


[5.511425495147705, 5.511425495147705]

In [34]:
model_2.evaluate(x_test,y_test)

25/25 [==============================] - 0s 1ms/step - loss: 5.5543 - mae: 5.5543


[5.554337024688721, 5.554337024688721]

### Beautiful, the loss function dropped from 8.6 to 5.5, which means that the model performs better.

#### step number 2: let's add some hidden layers.

In [49]:
# creating a simple neural network.

# Set random seed
tf.random.set_seed(42)

# Create a  model_3
model_3 = tf.keras.Sequential([
  tf.keras.layers.Dense(128),
  tf.keras.layers.Dense(64),
  
])

# Compile the model
model_3.compile(loss=tf.keras.losses.mae,
                        optimizer=tf.keras.optimizers.SGD(),
                        metrics=['mae'])

# Fit the model
model_3.fit(x_train, y_train, epochs=500, verbose=0)

In [50]:
#let's evaluate the model
model_3.evaluate(x_train,y_train)

99/99 [==============================] - 0s 772us/step - loss: 0.3254 - mae: 0.3254


[0.3254149556159973, 0.3254149556159973]

In [51]:
model_3.evaluate(x_test,y_test)

25/25 [==============================] - 0s 796us/step - loss: 0.3307 - mae: 0.3307


[0.330736368894577, 0.330736368894577]

## WoW! by adding two hidden layers, we got a huge improvment.

####Step3: lets fix the layer to one layer and change the optimizer to Adam from SGD to see the impact of optimizer.

In [52]:
# Set random seed
tf.random.set_seed(42)

# Create a  model_3
model_4 = tf.keras.Sequential([
  tf.keras.layers.Dense(128),
  
])

# Compile the model
model_4.compile(loss=tf.keras.losses.mae,
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=['mae'])

# Fit the model
model_4.fit(x_train, y_train, epochs=500, verbose=0)

In [55]:
#let's evaluate the model
model_4.evaluate(x_train,y_train)

99/99 [==============================] - 0s 827us/step - loss: 0.3248 - mae: 0.3248


[0.3247500956058502, 0.3247500956058502]

In [54]:
model_4.evaluate(x_test,y_test)

25/25 [==============================] - 0s 883us/step - loss: 0.3287 - mae: 0.3287


[0.3286767601966858, 0.3286767601966858]

### interesting, Adam works very well. Let's combine Adam and multi hidden layers

In [59]:
# Set random seed
tf.random.set_seed(42)

# Create a  model_5
model_5 = tf.keras.Sequential([
  tf.keras.layers.Dense(128),
  tf.keras.layers.Dense(64)
  
])

# Compile the model
model_5.compile(loss=tf.keras.losses.mae,
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=['mae'])

# Fit the model
model_5.fit(x_train, y_train, epochs=500, verbose=0)

In [60]:
model_5.evaluate(x_train,y_train)

99/99 [==============================] - 0s 828us/step - loss: 0.3463 - mae: 0.3463


[0.34630563855171204, 0.34630563855171204]

In [61]:
model_5.evaluate(x_test,y_test)

25/25 [==============================] - 0s 837us/step - loss: 0.3493 - mae: 0.3493


[0.3492981791496277, 0.3492981791496277]

### the combination of hyperparameters did not perform as well as the best model. Then, we can conclude that the most complex model is not the best model.